# Agent Behaviours (ABM)

In this Notebook all the behaviours that the spatial agents can perform are defined.  
<br/> The following Agent Behaviours are defined in the notebook:

## 1. Agent initial Position(Birth):
* Search(2D-3D)Space according to Stencil and the Space program and assigning the initial position
* Attraction / Repulsion movement of initial poostions of the agents based on the closeness relationships 

## 2. Occupy:
* (2D) random_occupy_squareness:
* (2D) random_occupy_cubish: 
* (2D) random_occupy_cubish_von_neumann_:
* (3D) one_neighbour_occupy_squareness_von_neumann:
* (3D) one_neighbour_occupy_squareness_moore:
* (3D) one_neighbour_occupy_cubish_behaviour:
* (2D) argmax_occupy_von_neumann:
* (3D) one_neighbour_occupy_cubish:


## 3. Directed Growth:
* (2D),(3D) Attraction: The base lattice gets modified with additional values based on graph distances between the selected agents
   making them grow towards each other
* (2D),(3D) Repulsion: The base lattice gets modified with additional values based on graph distances between the selected agents making them grow away from each other

## 4. Negotiation:

## 5. Unoccupy:

# Stencil Types

* (3D) Von_neumann
* (2D) Von_neumann_2d
* (3D) Moore
* (2D) Full_Floor
* (3D) Full_Lattice
* (2D) Squareness
* (3D) Cuboid


##  Initialization
Loading the necessary Libraries for the notebook

In [1]:
import os
import sys
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
import pickle

## Loading the Base Lattice ( Availability Lattice)
Since the stencil does not work for corner voxels the sides and the top and bottom voxels are made unavailable 

In [2]:
unit = 3
#Dummy Lattice for testing
Dim = 15
index = Dim -1
'''
bounds = np.ones((Dim,Dim), dtype=int)
bounds_2 = np.ones((Dim,Dim,Dim), dtype=int)
bounds[0,:] =0
bounds[:,0] =0
bounds[:,index] =0
bounds[index,:] =0
print (bounds)
init = np.tile(bounds,(Dim,1,1))
print(init.shape)
avail_lattice = tg.to_lattice(np.copy(init), init.shape)
init_avail_lattice = tg.to_lattice(np.copy(init), init.shape)
print (init_avail_lattice)
#print(avail_lattice.size)
'''

'\nbounds = np.ones((Dim,Dim), dtype=int)\nbounds_2 = np.ones((Dim,Dim,Dim), dtype=int)\nbounds[0,:] =0\nbounds[:,0] =0\nbounds[:,index] =0\nbounds[index,:] =0\nprint (bounds)\ninit = np.tile(bounds,(Dim,1,1))\nprint(init.shape)\navail_lattice = tg.to_lattice(np.copy(init), init.shape)\ninit_avail_lattice = tg.to_lattice(np.copy(init), init.shape)\nprint (init_avail_lattice)\n#print(avail_lattice.size)\n'

In [3]:
# loading the lattice from csv
lattice_path = os.path.relpath('Base_lattice.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)
#print(init_avail_lattice)

## Loading the Enviornment Lattices

In [4]:
#Dummy Lattice for testing

In [5]:
enviornment_lattice = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')/10
#print(enviornment_lattice)
#enviornment_lattice_2 = tg.to_lattice(1 - np.arange(bounds_2.size).reshape(bounds_2.shape) / bounds_2.size,bounds_2.shape )
#print(enviornment_lattice_2)

In [6]:
one_neighbour_factor = 1
two_neighbour_factor = 2
three_neighbour_factor = 3
four_neighbour_factor = 4

## Define the Neighborhoods for the behaviours (Stencils)

In [7]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
print(stencil_von_neumann)

[[[0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 1 0]
  [1 0 1]
  [0 1 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 0]]]


In [8]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [9]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [10]:
stencil_full_floor = tg.create_stencil("moore",int(Dim/2) )*0
stencil_full_floor[:,:,int(Dim/2)] = 1
stencil_full_floor.set_index([0,0,0], 0)
#print(stencil_full_floor)

In [11]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

In [12]:
stencil_full_lattice = tg.create_stencil("moore",int(Dim/2) )
stencil_full_lattice.set_index([0,0,0], 0)
#print(stencil_full_lattice)

## Vizualising the stencils 

In [13]:
Stencil_to_viz = stencil_squareness_t
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = np.array(Stencil_to_viz.shape) + 1
# The bottom left corner of the data set
grid.origin = [0,0,0]
# These are the cell sizes along each axis
grid.spacing = [1,1,1]

# Add the data values to the cell data
grid.cell_arrays["values"] = Stencil_to_viz.flatten(order="F")  # Flatten the stencil
threshed = grid.threshold([0.9, 1.1])

# adding the voxels: light red
p.add_mesh(threshed, show_edges=True, color="#ff8fa3", opacity=0.3)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(6.361720043489708, 6.361720043489708, 6.361720043489708),
 (1.5, 1.5, 1.5),
 (0.0, 0.0, 1.0)]

## Distance Field Construction for Attraction and Repulsion Behaviours 

In [14]:
'''
# find the number of all voxels
vox_count = avail_lattice.size 

# initialize the adjacency matrix
adj_mtrx = np.zeros((vox_count,vox_count))

# extract the neighbourhood of all voxels
# all_vox_neighs = avail_lattice.find_neighbours(stencil)

# Finding the index of the available voxels in avail_lattice
avail_index = np.array(np.where(avail_lattice == 1)).T

# fill the adjacency matrix using the list of all neighbours
for vox_loc in avail_index:
    # find the 1D id
    vox_id = np.ravel_multi_index(vox_loc, avail_lattice.shape)
    # retrieve the list of neighbours of the voxel based on the stencil
    vox_neighs = avail_lattice.find_neighbours_masked(stencil_von_neumann, loc = vox_loc)
    # iterating over the neighbours
    for neigh in vox_neighs:
        # setting the entry to one
        adj_mtrx[vox_id, neigh] = 1.0

# construct the graph 
g = nx.from_numpy_array(adj_mtrx)
'''

'\n# find the number of all voxels\nvox_count = avail_lattice.size \n\n# initialize the adjacency matrix\nadj_mtrx = np.zeros((vox_count,vox_count))\n\n# extract the neighbourhood of all voxels\n# all_vox_neighs = avail_lattice.find_neighbours(stencil)\n\n# Finding the index of the available voxels in avail_lattice\navail_index = np.array(np.where(avail_lattice == 1)).T\n\n# fill the adjacency matrix using the list of all neighbours\nfor vox_loc in avail_index:\n    # find the 1D id\n    vox_id = np.ravel_multi_index(vox_loc, avail_lattice.shape)\n    # retrieve the list of neighbours of the voxel based on the stencil\n    vox_neighs = avail_lattice.find_neighbours_masked(stencil_von_neumann, loc = vox_loc)\n    # iterating over the neighbours\n    for neigh in vox_neighs:\n        # setting the entry to one\n        adj_mtrx[vox_id, neigh] = 1.0\n\n# construct the graph \ng = nx.from_numpy_array(adj_mtrx)\n'

## Pickle Object

In [15]:
# compute the distance of all voxels to all voxels using floyd warshal algorithm
#dist_mtrx = nx.floyd_warshall_numpy(g)

In [16]:
#dist_matrix = pickle.dump( dist_mtrx, open( "dist_matrix.p", "wb" ) )

In [17]:
'''
#csv_flyod_warshall = np.savetxt('data.csv', dist_mtrx, delimiter=',')
#pickel 
#flyod_warshall_matrix =np.genfromtxt('data.csv',delimiter=',')
dist_matrix = pickle.load( open( "dist_matrix.p", "rb" ) ) 
flyod_warshall_matrix = dist_matrix
'''

'\n#csv_flyod_warshall = np.savetxt(\'data.csv\', dist_mtrx, delimiter=\',\')\n#pickel \n#flyod_warshall_matrix =np.genfromtxt(\'data.csv\',delimiter=\',\')\ndist_matrix = pickle.load( open( "dist_matrix.p", "rb" ) ) \nflyod_warshall_matrix = dist_matrix\n'

## Constructing Distance matrix to the agent origin

In [18]:
'''
# select the corresponding row in the matrix
myagent_attractor_one_dist = flyod_warshall_matrix[100]

# find the maximum valid value
max_valid = np.ma.masked_invalid(myagent_attractor_one_dist).max()
print(max_valid)
# set the infinities to one more than the maximum valid values
myagent_attractor_one_dist[myagent_attractor_one_dist == np.inf] = max_valid + 1

# mapping the values from (0, max) to (1, 0)
dist_flat_one = 1 - myagent_attractor_one_dist / np.max(myagent_attractor_one_dist)
#print(dist_flat_one)
# constructing the lattice
ent_acc_lattice_1 = tg.to_lattice(dist_flat_one.reshape(avail_lattice.shape), avail_lattice)
'''

'\n# select the corresponding row in the matrix\nmyagent_attractor_one_dist = flyod_warshall_matrix[100]\n\n# find the maximum valid value\nmax_valid = np.ma.masked_invalid(myagent_attractor_one_dist).max()\nprint(max_valid)\n# set the infinities to one more than the maximum valid values\nmyagent_attractor_one_dist[myagent_attractor_one_dist == np.inf] = max_valid + 1\n\n# mapping the values from (0, max) to (1, 0)\ndist_flat_one = 1 - myagent_attractor_one_dist / np.max(myagent_attractor_one_dist)\n#print(dist_flat_one)\n# constructing the lattice\nent_acc_lattice_1 = tg.to_lattice(dist_flat_one.reshape(avail_lattice.shape), avail_lattice)\n'

In [19]:
'''
# select the corresponding row in the matrix
myagent_attractor_two_dist = flyod_warshall_matrix[1200]

# find the maximum valid value
max_valid = np.ma.masked_invalid(myagent_attractor_two_dist).max()
#print(max_valid)
# set the infinities to one more than the maximum valid values
myagent_attractor_two_dist[myagent_attractor_two_dist == np.inf] = max_valid + 1

# mapping the values from (0, max) to (1, 0)
dist_flat_two = 1 - myagent_attractor_two_dist / np.max(myagent_attractor_two_dist)
#print(dist_flat_two)
# constructing the lattice
ent_acc_lattice_2 = tg.to_lattice(dist_flat_two.reshape(avail_lattice.shape), avail_lattice)
'''

'\n# select the corresponding row in the matrix\nmyagent_attractor_two_dist = flyod_warshall_matrix[1200]\n\n# find the maximum valid value\nmax_valid = np.ma.masked_invalid(myagent_attractor_two_dist).max()\n#print(max_valid)\n# set the infinities to one more than the maximum valid values\nmyagent_attractor_two_dist[myagent_attractor_two_dist == np.inf] = max_valid + 1\n\n# mapping the values from (0, max) to (1, 0)\ndist_flat_two = 1 - myagent_attractor_two_dist / np.max(myagent_attractor_two_dist)\n#print(dist_flat_two)\n# constructing the lattice\nent_acc_lattice_2 = tg.to_lattice(dist_flat_two.reshape(avail_lattice.shape), avail_lattice)\n'

In [20]:
'''
# select the corresponding row in the matrix
myagent_attractor_three_dist = flyod_warshall_matrix[100]
#print(myagent_attractor_three_dist)
# find the maximum valid value
max_valid = np.ma.masked_invalid(myagent_attractor_three_dist).max()

# set the infinities to one more than the maximum valid values
myagent_attractor_three_dist[myagent_attractor_three_dist == np.inf] = max_valid + 1

# mapping the values from (0, max) to (1, 0)
dist_flat_three = 1 - myagent_attractor_three_dist / np.max(myagent_attractor_three_dist)
#print(dist_flat_three)
# constructing the lattice
ent_acc_lattice_3 = tg.to_lattice(dist_flat_three.reshape(avail_lattice.shape), avail_lattice)
'''

'\n# select the corresponding row in the matrix\nmyagent_attractor_three_dist = flyod_warshall_matrix[100]\n#print(myagent_attractor_three_dist)\n# find the maximum valid value\nmax_valid = np.ma.masked_invalid(myagent_attractor_three_dist).max()\n\n# set the infinities to one more than the maximum valid values\nmyagent_attractor_three_dist[myagent_attractor_three_dist == np.inf] = max_valid + 1\n\n# mapping the values from (0, max) to (1, 0)\ndist_flat_three = 1 - myagent_attractor_three_dist / np.max(myagent_attractor_three_dist)\n#print(dist_flat_three)\n# constructing the lattice\nent_acc_lattice_3 = tg.to_lattice(dist_flat_three.reshape(avail_lattice.shape), avail_lattice)\n'

##  Agents

In the agent based system, when the stencil reaches its limit the full lattice/full floor depending on 2D or 3D  is considered in the simulation as a stencil.

### Define the Agents Class

In [21]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value)
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        #print(self.origin)
        

## Initializing Agents 

In [22]:
# occ_lattice intilization
occ_lattice = avail_lattice*0 -1
#print(occ_lattice)
# Finding the index of the available voxels in avail_lattice
avail_flat = avail_lattice.flatten()
avail_index = np.array(np.where(avail_lattice == 1)).T

# Randomly choosing one available voxels
agn_num = 1


In [23]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent
    

In [24]:
#Agent_2D = Agent_initilization 
Agent_one=initialize_agents_random_origin (stencil_von_neumann,avail_lattice )
Agent_two=initialize_agents_random_origin (stencil_von_neumann,avail_lattice )
Agent_three=initialize_agents_random_origin (stencil_von_neumann,avail_lattice )
Agent_four=initialize_agents_random_origin (stencil_von_neumann,avail_lattice )
Agent_five=initialize_agents_random_origin (stencil_von_neumann,avail_lattice )
Agent_six=initialize_agents_random_origin (stencil_von_neumann,avail_lattice )
Agent_seven=initialize_agents_random_origin (stencil_von_neumann,avail_lattice )



## Define Environment Class

In [26]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

In [27]:
#print(avail_lattice*1)

## Creating the Enviornment

In [28]:
# name the lattices myagent_attractor_one
env_lattices = {"availibility": avail_lattice,"enviornment": enviornment_lattice}
env_lattices_attractors_one = {"availibility": avail_lattice,"enviornment": enviornment_lattice}
env_lattices_attractors_two = {"availibility": avail_lattice,"enviornment": enviornment_lattice}
# initiate the environment
env = environment(env_lattices, Agent_one,50,"one_neighbour_occupy_cubish_behaviour")
env_2 = environment(env_lattices, Agent_two,100,"one_neighbour_occupy_cubish_behaviour")
env_3 = environment(env_lattices, Agent_three,50,"one_neighbour_occupy_cubish_behaviour")
env_4 = environment(env_lattices, Agent_four,200,"one_neighbour_occupy_cubish_behaviour")
env_5 = environment(env_lattices, Agent_five,100,"one_neighbour_occupy_cubish_behaviour")
env_6 = environment(env_lattices, Agent_six,500,"one_neighbour_occupy_cubish_behaviour")
env_7 = environment(env_lattices, Agent_seven,10,"one_neighbour_occupy_cubish_behaviour")

## Run the Simulation

In [29]:
#env,env_2,env_3,env_4,env_5,env_7,env,env_2,env_3,env_4,env_5
env_availability_viz = []
env_list =[env_6,env,env_2,env_3,env_4,env_5,env_7]
number_steps = max(map(lambda e:e.number_of_iterations,env_list))
for a in range(number_steps):
    # print(env.availibility)
    #print(env.agent_origin)

    for e in env_list:
        if a < e.number_of_iterations :
            #print(a)
            #print(e.number_of_iterations)
            if e.method_name == "one_neighbour_occupy_squareness_moore":
                e.one_neighbour_occupy_squareness_moore()
                
            elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                e.one_neighbour_occupy_cubish_agents()
                
            elif e.method_name == "random_occupy_squareness_agents" :
                e.random_occupy_squareness_agents()
            
            elif e.method_name == "random_occupy_cubish_agents" :
                e.random_occupy_cubish_agents()  
                
            elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                e.random_occupy_cubish_von_neumann_agents()                           
                
            elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                e.one_neighbour_occupy_squareness_von_neumann()                
                
                
            elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                e.one_neighbour_occupy_squareness_behaviour()  
                
            elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                e.one_neighbour_occupy_cubish_behaviour()  
            
            elif e.method_name == "argmax_occupy_von_neumann" :
                e.argmax_occupy_von_neumann()  
                
            env_availability_viz.append(e.availibility-1)
    

In [30]:
print(env_availability_viz[800])

[[[  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
     -1   -1]
  [  -1   -1   -1   -1   -1   -1   -1   -1 

## Vizualize the Simulation

In [31]:
p = pv.Plotter(notebook=True)

base_lattice = env_availability_viz[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.0, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = sum(map(lambda e:e.number_of_iterations,env_list))
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)


[3 3 3]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(372.9274879961902, 309.9274879961902, 243.92748799619022),
 (106.5, 43.5, -22.5),
 (0.0, 0.0, 1.0)]

## Empty Cells Resolution ??

## Questions?

1. How can I keep the same stencil for a certain number of iterations ( cube by cube occupancy)
2. Attraction making the distance lattices our of the two seeds of the agents and then occupying ?
3. Repulsion is invert of the distance lattices 
4. How can I make the burst behaviour ( also burst with static stencil iterations )
5. How can I reseed in burst 
6. Calculation of distance to multiple cells ( from a facade for example : Do you extract distance matrix to all cells in the facade and choose the minimum value for each cell from the distance matrices?)

## Saving the Lattices in csv

In [ ]:
# how much they are satisfyed with each othe
